In [8]:
import pandas as pd
import numpy as np

#df = pd.read_csv('~/Downloads/train_redux2_QC_05_000001.csv')
df = pd.read_csv('E:\\Downloads\\train_redux2.csv')
df.apply(pd.to_numeric,downcast='unsigned')
#y = pd.read_csv('~/Downloads/y2.csv', sep=';', usecols=[1])
y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1])
y.apply(pd.to_numeric,downcast='unsigned')
df.shape

(26495, 18124)

In [9]:
from sklearn.feature_selection import RFE
import xgboost as xgb

classifier = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1, random_state=42)
rfe = RFE(classifier, step=100, verbose=True, n_features_to_select=6100)
rfe.fit(df, y.values.ravel())

Fitting estimator with 18124 features.
Fitting estimator with 18024 features.
Fitting estimator with 17924 features.
Fitting estimator with 17824 features.
Fitting estimator with 17724 features.
Fitting estimator with 17624 features.
Fitting estimator with 17524 features.
Fitting estimator with 17424 features.
Fitting estimator with 17324 features.
Fitting estimator with 17224 features.
Fitting estimator with 17124 features.
Fitting estimator with 17024 features.
Fitting estimator with 16924 features.
Fitting estimator with 16824 features.
Fitting estimator with 16724 features.
Fitting estimator with 16624 features.
Fitting estimator with 16524 features.


KeyboardInterrupt: 

In [ ]:
ranks = rank_to_dict(map(float, rfe.ranking_), names, order=-1)

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2_selector = SelectKBest(chi2, k=7000)
df_kbest = chi2_selector.fit_transform(df, y)
export = pd.DataFrame(df_kbest)
export.to_csv('E:\\Downloads\\kbest2_7000.csv' , index=False)

In [4]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

Assoc = pd.DataFrame(columns={'SNP', 'assoc'})
SNP = pd.DataFrame(columns={'SNP'})
SNP['SNP'] = df.columns.values.ravel()
highAssoc = []
treshold = 0.05 / df.shape[1]

for i in range(df.shape[1]):
    model = LogisticRegression(C=1000000000)
    x = df.iloc[:, i].values.reshape(-1,1)
    model.fit(x, y.values.ravel())
    test = logit_pvalue(model, x)[1]
#    highAssoc.append(df.columns.values[i])
    highAssoc.append(test)
#    if test < treshold:
#        highAssoc.append(df.columns.values[i])
        #print(logit_pvalue(model, x)[1])
        
print("Total features: " + str(df.shape[1]))
print('found logit P value: ' + str(len(highAssoc)))
Assoc['SNP'] = SNP
Assoc['assoc'] = highAssoc
Assoc.to_csv('E:\\Downloads\\FS_assoc.csv', index=False)

Total features: 16004
found logit P value: 16004


In [7]:
import pandas as pd
import numpy as np

Assoc = pd.read_csv('E:\\Downloads\\FS_assoc.csv')
threshold = 0.0005 / 26482
result = Assoc[Assoc.assoc > threshold]
name = 'E:\\Downloads\\drop_FS_assoc_0005.csv'
result.to_csv(name , index=False)
result.shape

(9935, 2)